## Do not run

In [6]:
import torch
from torch import nn

In [7]:
train_words = []
with open("train_task.txt") as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line.split(" ")[0])

SOA
SOS
Cardinal
Theodore
McCarrick
:
Yet
Another
Fruit
of
Vatican
II
Still
,
we
’re
also
averse
to
speaking


In [8]:
matrix_len = len(target_vocab)
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(target_vocab):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

NameError: name 'target_vocab' is not defined

In [ ]:
embedding = nn.Embedding.from_pretrained("/home/raghavan/Downloads/wiki-news-300d-1M-subword.vec")

In [ ]:
from transformers import BertTokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

NameError: name 'BertTokenizer' is not defined

In [5]:
all_words = []
with open("all_task.txt") as f:
    all_lines = f.readlines()
    for line in all_lines:
        all_words.append(line.split(" ")[0])

In [7]:
all_words = list(set(all_words))

In [8]:
not_toknizable = []
for word in all_words:
    word_tokens = tokenizer.tokenize(word.lower())
    tokiziable = True
    for w in word_tokens:
        if "##" in w:
            tokiziable = False
    if not tokiziable:
        not_toknizable.append(word)

In [9]:
len(not_toknizable)

8943

In [12]:
not_toknizable[:100]

['Soetoro\n',
 'Upend\n',
 'presser',
 'dismissive',
 'Consortiumnews.com',
 'sneaky',
 'prosecuting\n',
 'Latina',
 'epochal',
 '701',
 'preventative',
 'pic.twitter.com/xAVa9TJW70\n',
 '2,037',
 'multiculturalism',
 'expeditiously',
 'gratuitous',
 'episcopate',
 'malfeasance',
 'deplorables',
 'defaming',
 'Arabiya',
 'imbecilic\n',
 'alleges',
 'analysing',
 'Awan',
 'nomad\n',
 'Penkoksi',
 'virgins',
 'appeasing',
 'Assata\n',
 'revel',
 'cooperating',
 'servitors',
 'implicate',
 'aggresses',
 'responsum',
 'tactfully\n',
 'Mahony',
 'parsec',
 'withstood',
 'unclued\n',
 'Impersonate\n',
 'quotation',
 'pic.twitter.com/z3JRAE80z6',
 'graver',
 'sideshow',
 'Kornmesser',
 'https://t.co/MC4Tlvy9gi',
 'whistles',
 'geniuses',
 'Nader',
 'incompetents',
 'BuzzFeed\n',
 'Bans',
 'Baldiserri',
 'Ranchers',
 'religiously\n',
 '@PrisonPlanet',
 'wield',
 '7-JB',
 'fecklessness',
 'Trumbo',
 'Sarkar',
 'incarcerate\n',
 'recourses',
 'Atta',
 'unfold',
 'Maurizio',
 'Ipsos',
 'backslapp

In [11]:
has_new_line = 0
for w in not_toknizable:
    if "\n" in w:
        has_new_line = has_new_line + 1

In [22]:
has_new_line

1363

In [23]:
'Langenberg\n'.endswith("\n")

True

In [24]:
'\n'.endswith("\n")

True

In [25]:
len("\n")

1

## Traning RNN

In [9]:
all_words = []
with open("all_task.txt") as f:
    all_lines = f.readlines()
    for line in all_lines:
        all_words.append(line.split(" ")[0])

In [10]:
len(all_words)

476736

In [11]:
import bcolz
import numpy as np

words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir="/home/raghavan/Downloads/fasttext", mode='w')

with open("/home/raghavan/Downloads/wiki-news-300d-1M-subword.vec", 'rb') as f:
    for l in f:
        line = l.decode().split()
        if line[0] != '999994':
            word = line[0]
            words.append(word)
            word2idx[word] = idx
            idx += 1
            vect = np.array(line[1:]).astype(np.float)
#             print(len(vect))
#             print(vect)
#             print(a)
            vectors.append(vect)


KeyboardInterrupt: 

In [ ]:
len(vectors)

In [ ]:
import pickle    
vectors = bcolz.carray(vectors[1:].reshape((-1, 300)), rootdir="/home/raghavan/Downloads/fasttext/6B.50.dat", mode='w')
vectors.flush()
pickle.dump(words, open(f'/home/raghavan/Downloads/fasttext/6B.50_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'/home/raghavan/Downloads/fasttext/6B.50_idx.pkl', 'wb'))

In [ ]:
glove_path = "/home/raghavan/Downloads/fasttext/"
vectors = bcolz.open(f'{glove_path}/6B.50.dat')[:]
words = pickle.load(open(f'{glove_path}/6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open(f'{glove_path}/6B.50_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

### Create weight matrix for embedding layer

In [ ]:
no_embedding_count = 0
target_vocab = list(set(all_words))
matrix_len = len(target_vocab)
weights_matrix = np.zeros((matrix_len, 300))
words_found = 0
emb_dim = 300
word2idx = {}
for i, word in enumerate(target_vocab):
    try: 
        word_lower = word.lower()
        if (word_lower.endswith("\n")):
            word_lower = word_lower.replace("\n","")
        weights_matrix[i] = glove[word_lower]
        word2idx[word_lower] = i
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))
        no_embedding_count = no_embedding_count + 1
        word2idx[word_lower] = i

In [ ]:
word2idx[""]

### Create pytorch dataset Class

In [ ]:
device = "cuda"

from torch.utils.data import Dataset, DataLoader

class Seq2SeqDataset(Dataset):
    
    def __init__(self,train,dev):
        
        self.state = "train"
        
        self.train_data =train 
        self.dev_data = dev
        
        self.data = self.train_data
        
    def __getitem__(self,ix):
        
        return (torch.tensor(self.data[ix]["x_data"]).to(device),torch.tensor(self.data[ix]["y_data"]).to(device))
    
    def __len__(self):
        return len(self.data)
    
    def set_split(self,state):
        self.state = state
        if state == "train":
            self.data = self.train_data
        else:
            self.data = self.dev_data

### Create Utils functions to create dataset

In [ ]:
def get_data_from_file(file_path):
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("SOA"):
                pass
            elif line.startswith("SOS"):
                if words:
                    examples.append((words,labels))
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                processed_word = splits[0].lower()
                if processed_word.endswith("\n"):
                    processed_word = processed_word.replace("\n","")
                words.append(processed_word)
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
        if words:
            examples.append((words,labels))
    return examples        

In [ ]:
labels = {"I" : 1 , "O":0}
max_seq_length =300
def words_to_ids(word_label_tup,max_seq_length=300,pad_token=0):
    processed_tups = []
    for tup in word_label_tup:
        ids = []
        lbls = []
        for word in tup[0]:
            ids.append(word2idx[word])
        for lbl in tup[1]:
            lbls.append(labels[lbl])            
        if len(ids) < max_seq_length:
            padding_length = max_seq_length - len(ids)
            ids += ([pad_token] * padding_length)
            lbls += ([-100] * padding_length)
        processed_tups.append({"x_data":ids,"y_data":lbls})
    return processed_tups

In [ ]:
train_data = words_to_ids(get_data_from_file("train_task.txt"))
dev_data = words_to_ids(get_data_from_file("dev_task.txt"))

In [ ]:
print(len(dev_data))

### Create Model class

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

def create_emb_layer(weights_matrix, non_trainable=True):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.weight.data.copy_(torch.from_numpy(weights_matrix))
    if non_trainable:
        emb_layer.weight.requires_grad = True

    return emb_layer, num_embeddings, embedding_dim

# class RNN(nn.Module):
#     def __init__(self, weights_matrix, hidden_size, num_layers):
#         super(self).__init__()
#         self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)
        
#     def forward(self, inp, hidden):
#         return self.gru(self.embedding(inp), hidden)
    
#     def init_hidden(self, batch_size):
#         return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))
class LSTMTagger(nn.Module):

    def __init__(self, weights_matrix, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(300, hidden_dim, 3, batch_first=True,bidirectional=True)
        self.dropout = nn.Dropout(0.2)
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim *2, 1)

    def forward(self, sentence):
        embeds = self.embedding(sentence)
#         print(embeds.view(len(sentence), 300, -1).shape)
        lstm_out, a = self.lstm(embeds.view(len(sentence), 300, -1))
        k = lstm_out.contiguous().view(-1, 600)
        tag_space = self.hidden2tag(self.dropout(k))
        tag_scores = F.sigmoid(tag_space)
        return tag_scores

### Create Dataset object

In [ ]:
dataset = Seq2SeqDataset(train_data,None)
dataset_dev = Seq2SeqDataset(None,dev_data)
dataset_dev.set_split("dev")

In [ ]:
dataset.__getitem__(0)[1]

In [ ]:
dataset_dev.__getitem__(0)

In [ ]:
dataloader = DataLoader(dataset=dataset,batch_size=180,shuffle=False,drop_last=True)
dataloader_dev = DataLoader(dataset=dataset_dev,batch_size=180,shuffle=False,drop_last=True)

In [ ]:
(list(dataloader)[0][1] == -100).nonzero()

### Define few utils functions

In [ ]:
from sklearn.metrics import classification_report , f1_score, precision_score ,recall_score

# def generate_batches(dataset, batch_size, shuffle=True,device="0"):
#     dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
#                             shuffle=shuffle)

#     for data_d in dataloader:
#         out_data_dict = {}
#         for name, tensor in data_dict.items():
#             out_data_dict[name] = data_dict[name].to(device)
#         yield out_data_dict  

def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (y_pred>0.5).cpu().float()#.max(dim=1)[1]
    p = precision_score(y_target.cpu().numpy(),y_pred_indices.cpu().detach().numpy())
    r = recall_score(y_target.cpu().numpy(),y_pred_indices.cpu().detach().numpy())
    f1 = f1_score(y_target.cpu().numpy(),y_pred_indices.cpu().detach().numpy())
    return p , r, f1
    #n_correct = (y_pred.view(-1)>0.5).sum().cpu().float().item()
    #return n_correct / len(y_pred_indices) * 100

### Run Training 

In [ ]:
from torch.nn import BCELoss
from torch import optim

model = LSTMTagger(weights_matrix,300,300,2).to(device)

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)
for epoch in range(10):
    
    dataset.set_split('train')

    
    running_p = 0
    running_r = 0
    running_f1 = 0
    running_loss = 0
    model.train()
    
    
    for batch_index,batch in enumerate(dataloader):
        
        optimizer.zero_grad()
        model.zero_grad()
        
        y_pred = model(batch[0])
        
        actual_sentence_mask = (batch[1].view(-1) != -100)
        y_pred_mask = y_pred[actual_sentence_mask]
        target_mask = batch[1].view(-1)[actual_sentence_mask]

        loss = loss_fn(y_pred_mask,target_mask.float())
        loss_t = loss.item()

        loss.backward()
        optimizer.step()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        p , r , f1 = compute_accuracy(y_pred_mask,target_mask.float())
        running_p += (p - running_p) / (batch_index + 1)   
        running_r += (r - running_r) / (batch_index + 1) 
        running_f1 += (f1 - running_f1) / (batch_index + 1) 
    
    running_p_test = 0
    running_r_test = 0
    running_f1_test = 0
    running_loss_test = 0
    
    print("Training :Epoch: %s ,loss: %s ,Precision: %s, Recall %s , F1: %s" % (epoch,running_loss,running_p,running_r,running_f1))
    model.eval()
    for batch_index,batch in enumerate(dataloader_dev):
        
        y_pred = model(batch[0])
        
        actual_sentence_mask = (batch[1].view(-1) != -100)
        y_pred_mask = y_pred[actual_sentence_mask]
        target_mask = batch[1].view(-1)[actual_sentence_mask]

        loss = loss_fn(y_pred_mask,target_mask.float())
        loss_t = loss.item()

        running_loss_test += (loss_t - running_loss_test) / (batch_index + 1)

        p , r , f1 = compute_accuracy(y_pred_mask,target_mask.float())
        running_p_test += (p - running_p_test) / (batch_index + 1)   
        running_r_test += (r - running_r_test) / (batch_index + 1) 
        running_f1_test += (f1 - running_f1_test) / (batch_index + 1) 
    
    
    print("Dev:  Epoch: %s ,loss: %s ,Precision: %s, Recall %s , F1: %s" % (epoch,running_loss_test,running_p_test,running_r_test,running_f1_test))


In [ ]:
model.eval()

### Prep for test samples

In [ ]:
def get_data_from_file_for_test(file_path):
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        for line in f:
            if line.startswith("SOA"):
                pass
            elif line.startswith("SOS"):
                if words:
                    examples.append(words)
                    words = []
            else:
                splits = line.split(" ")
                processed_word = splits[0].lower()
                if processed_word.endswith("\n"):
                    processed_word = processed_word.replace("\n","")
                words.append(processed_word)
        if words:
            examples.append(words)
    return examples        

In [ ]:
labels = {"I" : 1 , "O":0}
max_seq_length =300
def words_to_ids_for_test(word_label_tup,max_seq_length=300,pad_token=0):
    processed_tups = []
    all_words = []
    for ws in word_label_tup:
        ids = []
        words = []
        for word in ws:
            ids.append(word2idx[word])    
            words.append(word)
        if len(ids) < max_seq_length:
            padding_length = max_seq_length - len(ids)
            ids += ([pad_token] * padding_length)
            #words += ([";lkjh"] * padding_length)
        all_words.append(words)
        processed_tups.append({"x_data":ids,"y_data":[]})
    return processed_tups, all_words

In [ ]:
test_data ,words = words_to_ids_for_test(get_data_from_file_for_test("test_task.txt"))

In [ ]:
get_data_from_file_for_test("test_task.txt")

In [ ]:
test_dataset = Seq2SeqDataset(None, test_data)
test_dataset.set_split("test")

In [ ]:
test_dataset.__getitem__(0)

In [ ]:
test_dataloader = DataLoader(dataset=test_dataset,batch_size=1,shuffle=False)

In [ ]:
model.eval()

In [ ]:
len(list(test_dataloader))

In [ ]:
sentence_and_prediction = []
for batch_index,batch in enumerate(test_dataloader):
    
    y_pred = model(batch[0])
    sentence_and_prediction.append((batch[0].detach().cpu().numpy(),y_pred.detach().cpu().numpy()))



In [ ]:
len(sentence_and_prediction)

In [ ]:
# inv_weight_matrix = {v: k for k, v in word2idx.items()}

# sentence_and_prediction_corrent_shape = []
# for sentence_prediction in sentence_and_prediction:
#     end_index = np.where(sentence_prediction[0][0] == 0)[0][0]
#     output_labels = []
#     for i in sentence_prediction[1][:end_index].reshape(-1):
#         if i > 0.5 :
#             label = "I"
#         else:
#             label = "O"
#         output_labels.append(label)
#     input_words = []
#     for i in sentence_prediction[0][0][:end_index]:
#         input_words.append(inv_weight_matrix[i])
        
#     sentence_and_prediction_corrent_shape.append((input_words,output_labels))
    

In [ ]:
predictions = []
for i in sentence_and_prediction:
    end_index = np.where(i[0][0] == 0)[0][0]
    for i in sentence_prediction[1][:end_index].reshape(-1):
        if i > 0.5 :
            label = "I"
        else:
            label = "O"
        predictions.append(label)

In [ ]:
article_strings = []
with open("test_task.txt", "r") as f:
    lines = f.readlines()
    for ix,line in list(enumerate(lines)):
#         print(line)
        if (line == '\n' and (lines[ix-1].startswith("EOA") or lines[ix-1].startswith("EOS")) and ix >1):
#         if line.startswith("SOA") or line.startswith("SOS"):
            pass
        else:
            article_strings.append(line)

In [ ]:
article_string_prediction = list(zip(article_strings,predictions))

In [ ]:
# article_string_prediction = []
# for tup in sentence_and_prediction_corrent_shape:
#     for i in zip(tup[0],tup[1]):
#         article_string_prediction.append(i)

In [ ]:
article_string_prediction[0]

In [ ]:
import os
final_article_string_prediciton = []
with open(os.path.join("test_task.txt"), "r") as f:
    lines = f.readlines()
    count = 0
    for ix, line in enumerate(lines):
        if (line == '\n' and (lines[ix-1].startswith("EOA") or lines[ix-1].startswith("EOS")) and ix >1):
            final_article_string_prediciton.append(("",""))
        else:
            final_article_string_prediciton.append((article_string_prediction[count][0].replace("\n",""),article_string_prediction[count][1].replace("\n","").replace("\n","")))
            count = count + 1

final_string_array = [ i[0] + " " + i[1]for i in final_article_string_prediciton]

final_string = "\n".join(final_string_array)

with open(os.path.join("final_prediction.txt"), "w") as f:
    f.writelines(final_string)


In [ ]:
# len(%precision)

In [ ]:
len(predictions)

In [ ]:
article_strings[:10]

In [ ]:
predictions[:10]